In [3]:
import subprocess
import os
import datetime
import ipywidgets as widgets
import pandas as pd
import numpy as np
import folium
from folium import plugins
import branca.colormap as cm
import matplotlib.pyplot as plt
import mplleaflet
from matplotlib.patches import Ellipse
import math

In [4]:
lat = widgets.FloatText(
    value=33.0,
    description='Lat',
)
lon = widgets.FloatText(
    value=-115.0,
    description='Lon',
)
width = widgets.FloatText(
    value=1,
    description='Width',
)
height = widgets.FloatText(
    value=1,
    description='Height')
t1 = widgets.DatePicker(description='Time 1',value = datetime.date(2010,4,7))
t2 = widgets.DatePicker(description='Time 2',value = datetime.date(2010,4,9))
file_name = widgets.Text(description='Output name', value='displacement')
all_widgets = [lat, lon, width, height, t1, t2, file_name]
widgets.GridBox(all_widgets, layout=widgets.Layout(
    grid_template_columns='repeat(2,400px)'))

GridBox(children=(FloatText(value=33.0, description='Lat'), FloatText(value=-115.0, description='Lon'), FloatT…

In [5]:
#disp_args = f'python getDisplacement.py --lat {lat.value} --lon {lon.value} --width {width.value} --height {height.value} -t1 {t1.value.strftime("%Y-%m-%d")} -t2 {t2.value.strftime("%Y-%m-%d")} -o {file_name.value}'
# parameters dict object
# required
parameters = {"lat":lat.value,"lon":lon.value,"width":width.value,"height":height.value,"epoch1":t1.value.strftime("%Y-%m-%d"),"epoch2":t2.value.strftime("%Y-%m-%d"),"output":file_name.value}
# optional
parameters['scale'] = None
parameters['ref'] = None 
parameters['ct'] = None 
parameters['pt'] = None
parameters['eon'] = True
parameters['mon'] = False
parameters['dwin1'] = None
parameters['dwin2'] = None
parameters['vabs'] = False

In [6]:
# turn dict object to a class object
class objdict(dict):

    def __getattr__(self, name):
        if name in self:
            return self[name]
        else:
            raise AttributeError("No such attribute: " + name)

    def __setattr__(self, name, value):
        self[name] = value

    def __delattr__(self, name):
        if name in self:
            del self[name]
        else:
            raise AttributeError("No such attribute: " + name)


In [7]:
#process = subprocess.getoutput(disp_args)
from getDisplacement_fix import getDisplacement

In [8]:
data_table = getDisplacement(objdict(parameters))

In [12]:
datadf = pd.DataFrame(data_table, columns=["Site","Lon","Lat","Delta E","Delta N","Delta V","Sigma E","Sigma N","Sigma V"])

In [13]:
datadf

,Site,Lon,Lat,Delta E,Delta N,Delta V,Sigma E,Sigma N,Sigma V
0,GMPK,-114.827347,33.051084,1.769741,-1.970751,-0.684817,0.313947,0.365123,1.364453
1,IID2,-115.031804,32.706168,7.359603,-4.292746,-0.992672,0.297097,0.350955,1.247817
2,P499,-115.487914,32.979606,1.931053,-9.400471,0.668385,0.291891,0.342312,1.219653
3,P500,-115.299933,32.690046,12.283892,-12.540173,0.438207,0.280181,0.336806,1.140766
4,P501,-115.397914,32.875756,3.109264,-11.023336,0.828574,0.287773,0.348138,1.191560
5,P502,-115.421913,32.982450,1.870836,-8.353152,1.136465,0.290019,0.352150,1.200679
6,P508,-115.428707,33.247780,0.354472,-4.670190,0.651837,0.288847,0.345983,1.187426
7,P509,-115.293919,32.890662,3.064828,-7.699427,0.606118,0.277898,0.334891,1.126954
8,P510,-115.343334,33.143574,0.552332,-5.217306,-0.144948,0.290733,0.348747,1.245531


In [27]:
def create_map(df, lon_col='Lon', lat_col='Lat', data_col='Delta V', site_col='Site'):
    '''
    Parameters
    ----------
    df :
        Pandas dataframe with column of lat, long, and a data column
    lon_col:
        Name of column with longitude coords. Default value of 'Lon'
    lat_col:
        Name of column with lattitude coords. Default value of 'Lat'
    data_col:
        Name of column with values to be colormapped. Default is 'Delta N'
    site_col:
        Name of column with GPS site names. 
    -------
    Returns:
        Folium map object
    '''
    start_cords = (df[lat_col].median(),
                   df[lon_col].median())
    my_map = folium.Map(start_cords,
                        tiles=None,zoom_start=8, width=750, height=465,contol_scale=True)
    plugins.Fullscreen(position='topleft').add_to(my_map)
    # load basemap with name
    folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
                        attr='Tiles &copy; Esri', name='World_Topo_Map').add_to(my_map)

    
    # making MPL seismic colormap in Branca
    colors_list = ['blue', 'white', 'red']
    minimum = df[data_col].min()
    maximum = df[data_col].max()
    colormap = cm.LinearColormap(
        colors=colors_list, vmin=minimum, vmax=maximum)
    stations = folium.FeatureGroup(
        name='Stations')
    vertical = folium.FeatureGroup(
        name='Vertical Displacement')
    horizontal = folium.FeatureGroup(
        name='Horizontal Displacement')
    # displace scale, mm/yer/deg
    scale = 320
    icon_url = "https://maps.google.com/mapfiles/kml/paddle/grn-blank.png"
    html = '<a target="_blank" href="https://sideshow.jpl.nasa.gov/post/links/{site}.html"><img src="https://sideshow.jpl.nasa.gov/post/plots/{site}.jpg" width="300" height="300"></a>'
    for cord, disp, site in zip(zip(df[lat_col], df[lon_col]), df[data_col], df[site_col]):
        folium.Circle(
            location=cord,
            radius=abs(disp)*scale,
            fill=True,
            color=colormap(disp),
            popup=f'Vertical Displacement:{"{:10.4f}".format(disp)} mm,   Site:{site}',
            stroke=True,
            weight=1,
            fill_opacity=.8
        ).add_to(vertical)
        icon = folium.features.CustomIcon(icon_url,icon_size=(16, 16),icon_anchor=(8,16))
        #iframe = folium.element.IFrame(html=html, width=500, height=300)
        s_html = html.format(site=site)
        popup = folium.Popup(s_html, max_width=2650)

        folium.Marker(
            location=cord,
            icon=icon,
            popup=popup,
        ).add_to(stations)
    
    horizontal = folium.FeatureGroup(
        name='Horizontal Displacement')
    for lat,lon,vlon,vlat in zip(df[lat_col], df[lon_col],df['Delta E'],df['Delta N']):
        linedis = [[lat,lon],[lat+vlat/scale,lon+vlon/scale/math.cos(lat*math.pi/180.)]]
        folium.PolyLine(
            locations=linedis,
            color='red',
            weight=2
        ).add_to(horizontal)
                   
    errorlayer = folium.FeatureGroup(
        name='Error eclipse')
    for lat,lon,vlon,vlat,slon,slat in zip(df[lat_col], df[lon_col],df['Delta E'],df['Delta N'],df['Sigma E'],df['Sigma N']):
        theta = 0
        coords = []
        for k in range(0,31):
            angle = k/30*2*math.pi
            elon = slon*math.cos(angle)*math.cos(theta)-slat*math.sin(angle)*math.sin(theta)
            elat = slon*math.cos(angle)*math.sin(theta)+slat*math.sin(angle)*math.cos(theta)
            elon = (elon+vlon)/scale/math.cos(lat*math.pi/180.)
            elat = (elat+vlat)/scale
            coords.append([lat+elat,lon+elon])
        folium.Polygon(
            locations=coords,
            fill=False,
            color="black",
            weight=2 
        ).add_to(errorlayer)
    my_map.add_child(stations)
    my_map.add_child(vertical)
    my_map.add_child(horizontal)
    my_map.add_child(errorlayer)
    my_map.add_child(colormap)
#     folium.raster_layers.ImageOverlay('error_elippse.png', [[df[lat_col].min(
#     ), df[lon_col].min()], [df[lat_col].max(), df[lon_col].max()]], name='Error Ellipse').add_to(my_map)
    my_map.add_child(folium.LayerControl(postion='bottomleft'))
    
    return my_map

In [28]:
# I have no idea why the arrows are plotting off center from the circles. Seems super inconsistent. 
create_map(datadf)